[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

# Patrones de *URLs*.

**ADVERTENCIA:**  

Para poder realizar exitosamente los ejercicios de esta *notebook*, es necesario haber seguido al pie de la letra y en orden sucesivo las instrucciones de todas las *notebooks* previas.

El módulo ```django.urls``` cuenta con herramientas que permiten identificar y filtrar *URLs*  de modo tal que pueda crear servicios dinámicos a partir de la información contenida en ellos.

La documentación correspondiente puede ser consultada en la siguiente liga:

https://docs.djangoproject.com/en/3.2/topics/http/urls/

## Identificación de segmentos en la ruta de una *URL*.

Previamente se estudió la funcionalidad básica de ```django.urls.path()``` realizando las siguientes operaciones:

* Identificando un patrón de *URL*.
* Llamando a una función de vista relacionada con el patrón.
* Ingresando automáticamente al objeto ```request``` como argumento de la función de vista.

Es convieniente recordar que la sintaxis de la función ```django.urls.path()``` es:

```
django.urls.path('<patrón>', <funcion de vista>, <kwargs>)
```

Donde:
* ```<patrón>``` es una cadena de cartacteres que debe de coincidir con la *URL*. 
* ```<función de vista>``` es la referencia a la función de vista al auq está relacionada el patrón.
* ```<kwargs>``` son otros argumentos expresados como ```<nombre>=<valor>```.

### Extracción de segmentos identificables en la ruta de una *URL*.

La función ```django.urls.path()``` puede:
* Identificar segmentos de caracteres delimitados por el caracter ```/```.
* Convertir cada cadena de caracteres segmentada en un tipo de dato específico.
* Asignarle un nombre a cada valor obtenido.

### Tipos de datos extraibles de un segmento de ruta de una *URL*:

Los tipos de datos en los que puede ser convertido un segmento son los siguiientes:

* ```string```: es el tipo de datos por defecto y corresponde a una cadena de caracteres.
* ```int```: corrsponde a un número entero positivo, incluyendo al cero.
* ```slug```: es una combinación de caracteres *ASCII*, guiones medios ```-``` y guiones bajos ```_```.
* ```uuid```: para las secuencias que se apegan a la especificación [*RFC 4122*](https://tools.ietf.org/html/rfc4122.html).
* ```path```: captura el segmento de la *URL* incluyendo las diagonales ```/```.

**Ejemplo:**

* La ruta ```main/53/azul/01``` puede ser segmentada obteniendo las siguientes cadenas de caracteres a partir de ```main/```:
    * ```'53'```.
    * ```'azul'```.
    * ```'01'```.
* Cada uno de estos segmentos puede ser convertido en un tipo compatible. Por defecto, todos los segmentos son de tipo ```string```.

### Definición del patrón de extracción de segmentos.

La cadena de caracteres que define a un patrón con segmentos extraibles tendría una sintaxis similar a la siguiente:

```
'<patrón inicial>/<segmento 1>/<segmento 2>/.../<segmento n>'
```

Donde:

* ```<patrón inicial>``` es un patrón específico que debe de coincidir con el primer segmento de la ruta.
* ```<segmento x>``` corresponde a cada segmento a extraer.

### Definición de nombre y tipo de cada segmento.

Es posible asignarle un nombre a cada segmento mediante la sintaxis ```tipo:nombre``` encerrado entre los signos de "menor que" ```<``` y "mayor que" ```>```.  

**Ejemplo:**
 * Para capturar los segmentos de la *URL* ```main/53/azul/01``` se usaría el siguiente patrón:
 
 ```
 'main/<int:codigo>/<string:color>/<int:cantidad>'
```
* Se obtendrían:
    * El objeto con nombre ```codigo```, de tipo ```int``` y con valor de ```53```.
    * El objeto con nombre ```color```, de tipo ```string``` y con valor de ```'azul'```.
    * El objeto con nombre ```cantidad```, de tipo ```ìnt``` y con valor de ```1```.   
    
Estos valores serían usados como argumentos adicionales a ```request``` en la función de vista correspondiente.

## Identificación de rutas con expresiones regulares.

La función ```django.url.re_path()``` permite hacer búsquedas que coincidan con una expresión regular.

```
re_path(r'<expresión regular>', <función de vista>)
```
En caso de que la expresión regular extraiga datos, estos datos se ingresarán como argumentos después del objeto ```request```. 


## Ejemplo ilustrativo.

### El *script* ```src/07/urls.py```.

El *script* ```src/07/urls.py``` contiene lo siguiente:


``` python
from django.urls import path, re_path 
from . import views

urlpatterns = [path('', views.index, name="inicio"), 
               path('vista', views.vista),
               re_path(r'^claves/(?P<clave>[0-9]{4}$)', views.clave),
               path('claves/<int:numero>', views.numero),
               path('claves/<str:nombre>', views.saluda),]
```

* En caso de que se ingrese una *URL* con exactamente 4 dígitos después de ```main/claves/```, se ejecutará la función ```views.clave()``` y se enviarán los 4 dígitos como segundo argumento.
* En caso de que se ingrese una *URL* con números que no sean de 4 dígitos después de ```main/claves/```, se ejecutará la función ```views.numero()``` y se enviará el número entero como segundo argumento.
* En caso de que se ingrese una *URL* con algo que no cumpla las condiciones previas después de ```main/claves/```, se ejecutará la función ```views.saluda()``` y se enviará el contenido como segundo argumento.

* A continuación se sustituirá al *script* ```tutorial/main/urls.py```  con el *script* ```src/07/urls.py```.

* Para la plataforma *GNU/Linux* y *MacOS X*.

In [ ]:
!cp src/07/urls.py tutorial/main/urls.py 

* Para la plataforma *Windows*.

In [ ]:
!copy src\07\urls.py tutorial\main\urls.py 

* La siguiente celda desplegará las modificaciones al *script* ```tutorial/main/urls.py```.

In [ ]:
%pycat tutorial/main/urls.py

### El *script* ```src/07/views.py```.

El *script* ```src/07/views.py``` contiene el siguiente código:

``` python
from django.http import HttpResponse


def index(request):
    return HttpResponse("<h1>Hola, mundo.</h1>")


def vista(request):
    return HttpResponse('<ul><li>URL: {}</li><li>Método: {}</li><li>Codificación: {}</li><li>Argumentos: {}</li></ul>'.format(request.path, request.method, request.encoding, request.GET.dict()))


def clave(request, clave):
    return HttpResponse('<h1>Ingresaste la clave: {}</h1>'.format(str(clave)))


def numero(request, numero):
    return HttpResponse('<h1>Ingresaste el número: {}</h1>'.format(str(numero)))


def saluda(request, nombre):
    return HttpResponse('<h1>¡Hola, {}!</h1>'.format(nombre))
```

* A continuación se sustituirá al *script* ```tutorial/main/views.py```  con el *script* ```src/07/views.py```.

* Para las plataformas *GNU/Linux* y *MacOS X*.

In [ ]:
!cp src/07/views.py tutorial/main/views.py

* Para la plataforma *Windows*.

In [ ]:
!copy src\07\views.py tutorial\main\views.py 

* La siguiente celda desplegará las modificaciones al *script* ```tutorial/main/views.py```.

In [ ]:
%pycat tutorial/main/views.py

## Arranque del servicio.

### Arranque desde la *notebook*.

In [ ]:
%cd tutorial

In [ ]:
!python manage.py runserver 0.0.0.0:8000

### Arranque desde una terminal.

#### Carga del entorno.

* En caso de que se acceda a la terminal de la máquina virtual proporcionada por *Pythonista<sup>®</sup>*, es necesario cargar el entormno con el siguiente comando.

```
source ~/pythonista/bin/activate
```
#### Inicio del servidor.

* Desde la terminal ubicar el *shell* en el directorio ```tutorial```, en el cual se encuentra el *script* ```manage.py```.

```
python manage.py runserver 0.0.0.0:8000
```

**Nota:** 
Es necesario que el *firewall* de su equipo esté configurado para transmitir desde el puerto ```8000```. 

## Resultados.

* La *URL* http://localhost:8000/main/claves/12345678 ejecutará la función ```tutorial.main.views.numero()``` en vista de que el segmento posterior a ```main/claves/``` contiene un número de 8 dígitos.

<img src="imagenes/07/clave-12345678.png" width="600">

* La *URL* http://localhost:8000/main/claves/12 ejecutará la función ```tutorial.main.views.numero()``` en vista de que el segmento posterior a ```main/claves/``` contiene un número de 2 dígitos.

<img src="imagenes/07/clave-12.png" width="600">

* La *URL* http://localhost:8000/main/claves/1234 ejecutará la función ```tutorial.main.views.clave()``` en vista de que el segmento posterior a ```main/claves/``` contiene un número de 4 dígitos.

<img src="imagenes/07/clave-1234.png" width="600">

* La *URL* *http://localhost:8000/main/claves/Juan* ejecutará la función ```tutorial.main.views.saluda()``` en vista de que el segmento posterior a ```main/claves/``` contiene una cadena de caracteres.

<img src="imagenes/07/clave-juan.png" width="600">

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>